In [2]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [3]:
Nx = Ny = 5
alpha = 1/5
Tx = Ty = 0
periodicity = 0
HSP_T(Nx, Ny, alpha, Tx, Ty) == Hofstadter_SP(Nx, Ny, alpha, periodicity) #check

true

In [8]:
using LinearAlgebra

In [9]:
E0, U0 = eigen(HSP_T(3, 3, 1/3, 1, 1))

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
9-element Vector{Float64}:
 -2.6128790823021095
 -2.612879082302096
 -2.6128790823020873
  0.36854446388968815
  0.3685444638896898
  0.36854446388969064
  2.2443346184124118
  2.244334618412412
  2.2443346184124127
vectors:
9×9 Matrix{ComplexF64}:
 -0.346293-0.152583im  -0.0598026+0.103581im     …   0.211699+0.394462im
 0.0876357-0.278051im   0.0646229+0.100644im        -0.192146-0.324522im
 0.0277132-0.379545im  -0.0835113+0.381297im         0.250771+0.09372im
 -0.316786+0.271231im   -0.119472-0.00564297im      -0.134815-0.460673im
 -0.083809+0.299801im    0.119605+2.94018e-17im     0.0961471+0.0178297im
 -0.316032+0.13225im     0.275729-0.276288im     …  -0.163401+0.27115im
 -0.338153+0.224088im   -0.288457+0.262971im        -0.145154+0.337612im
 -0.251271+0.10533im    -0.377137+0.100644im        0.0142792-0.332031im
       0.0+0.0im         -0.57735-0.0im                   0.0+0.0im

In [14]:
# There is degeneracy which is needed 
i = sortperm(E0);println(i)
sorted_U = U0[:,i]

[1, 2, 3, 4, 5, 6, 7, 8, 9]


9×9 Matrix{ComplexF64}:
 -0.346293-0.152583im  -0.0598026+0.103581im     …   0.211699+0.394462im
 0.0876357-0.278051im   0.0646229+0.100644im        -0.192146-0.324522im
 0.0277132-0.379545im  -0.0835113+0.381297im         0.250771+0.09372im
 -0.316786+0.271231im   -0.119472-0.00564297im      -0.134815-0.460673im
 -0.083809+0.299801im    0.119605+2.94018e-17im     0.0961471+0.0178297im
 -0.316032+0.13225im     0.275729-0.276288im     …  -0.163401+0.27115im
 -0.338153+0.224088im   -0.288457+0.262971im        -0.145154+0.337612im
 -0.251271+0.10533im    -0.377137+0.100644im        0.0142792-0.332031im
       0.0+0.0im         -0.57735-0.0im                   0.0+0.0im

In [21]:
# Twisted Angle Space for Calculating Real Space Chern Numbers
Tsize=25
dx=2*pi/Tsize
dy=dx
Tx=collect(range(start=0, stop=2*pi, step=dx))
Ty=collect(range(start=0, stop=2*pi, step=dy))

25-element Vector{Float64}:
 0.0
 0.25132741228718347
 0.5026548245743669
 0.7539822368615504
 1.0053096491487339
 1.2566370614359172
 1.5079644737231008
 1.7592918860102844
 2.0106192982974678
 2.261946710584651
 2.5132741228718345
 2.7646015351590183
 3.0159289474462017
 3.267256359733385
 3.518583772020569
 3.769911184307752
 4.0212385965949355
 4.272566008882119
 4.523893421169302
 4.775220833456486
 5.026548245743669
 5.277875658030853
 5.529203070318037
 5.7805304826052195
 6.031857894892403

In [22]:
E0

9-element Vector{Float64}:
 -2.6128790823021095
 -2.612879082302096
 -2.6128790823020873
  0.36854446388968815
  0.3685444638896898
  0.36854446388969064
  2.2443346184124118
  2.244334618412412
  2.2443346184124127

In [29]:
function Chern_Nums(Nx, Ny, alpha, Tx, Ty, n1, n2)
    
    Sum = 0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            
            w1, v1 = eigen(HSP_T(Nx, Ny, alpha, Tx[tx], Ty[ty]))
            v1 = v1[:,n1:n2]
            #------------------------------------
            w2, v2 = eigen(HSP_T(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty]))
            v2 = v2[:,n1:n2]
            #------------------------------------
            w3, v3 = eigen(HSP_T(Nx, Ny, alpha, Tx[tx], Ty[ty]+dy))
            v3 = v3[:,n1:n2]
            #------------------------------------
            w4, v4 = eigen(HSP_T(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty]+dy))
            v4 = v4[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(*(adjoint(v1),v2))
            U1=U1/abs(U1)
            U2=det(*(adjoint(v2),v4))
            U2=U2/abs(U2)
            U3=det(*(adjoint(v3),v4))
            U3=U3/abs(U3)
            U4=det(*(adjoint(v1),v3))
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum
            Sum=Sum+F
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

Chern_Nums(3, 3, 1/3, Tx, Ty, 7, 9)

1.0000000000000018 - 3.300003801258741e-16im